In [2]:
import numpy as np
import random
import copy
import sys

# ↓問題生成

In [1]:
def puzzle_generation(puzzle_board, issue):
    #ファイル読み込み
    f = open(issue, 'r', encoding='UTF-8')
    data = f.readlines()
    f.close()


    #問題フォーマットの1行目の読み取り ----------------------------------------------------------------------------------------
    s0 = data[0].rstrip('\n').rstrip(',')
    l0 = [int(x.strip()) for x in s0.split(',')]

    colmun = l0[0]              #列
    row = l0[1]                 #行
    selection_cost = l0[2]      #選択コスト


    
    #スライドパズルの盤面の読み取り ------------------------------------------------------------------------------------------
    # for i in range(1, row+1):
    #     row_data = data[i].rstrip('\n').rstrip(',')
    #     line_data = [int(x.strip()) for x in row_data.split(',')]
    #     row_array = np.array(line_data)
    #     if i == 1:
    #         puzzle_board = row_array
    #     else:
    #         puzzle_board = np.vstack((puzzle_board, row_array))

    colmun = 6
    row = 6
    #ランダム生成 ---------------------------------------------------------------------------------------------------------
    random_num = random.sample(range(1, row*colmun+1), row*colmun)
    random_list = np.array(random_num)
    puzzle_board = random_list.reshape(row, colmun)

    copy_board = puzzle_board.copy()

    return puzzle_board, colmun, row, selection_cost, copy_board

# ↓移動関数

In [3]:
#選択画像を上に移動
def up(target_value, selection_value, exchange_operation, fixed_value):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[0] != 0 and (puzzle_board[selection_coord[0]-1][selection_coord[1]] in fixed_value) == False:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]-1][selection_coord[1]])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]-colmun], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'U'

        # print()
        # print(puzzle_board)
        # print("       ↓")

        return target_coord, selection_coord, exchange_operation

    
    else:
        return sys.exit("この上は盤面外、もしくは並び替えが完了しているよ!")

In [4]:
#選択画像を下に移動
def down(target_value, selection_value, exchange_operation, fixed_value):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[0] != colmun-1 and (puzzle_board[selection_coord[0]+1][selection_coord[1]] in fixed_value) == False:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]+1][selection_coord[1]])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]+colmun], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'D'

        # print()
        # print(puzzle_board)
        # print("       ↓")

        return target_coord, selection_coord, exchange_operation

    else:
        return sys.exit("この下は盤面外、もしくは並び替えが完了しているよ!")

In [5]:
#選択画像を左に移動
def left(target_value, selection_value, exchange_operation, fixed_value):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[1] != 0 and (puzzle_board[selection_coord[0]][selection_coord[1]-1] in fixed_value) == False:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]][selection_coord[1]-1])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]-1], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'L'

        # print()
        # print(puzzle_board)
        # print("       ↓")

        return target_coord, selection_coord, exchange_operation

    else:
        return sys.exit("この左は盤面外、もしくは並び替えが完了しているよ!")

In [6]:
#選択画像を右に移動
def right(target_value, selection_value, exchange_operation, fixed_value):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[1] != colmun-1 and (puzzle_board[selection_coord[0]][selection_coord[1]+1] in fixed_value) == False:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]][selection_coord[1]+1])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]+1], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'R'

        # print()
        # print(puzzle_board)
        # print("       ↓")

        return target_coord, selection_coord, exchange_operation

    else:
        return sys.exit("この右は盤面外、もしくは並び替えが完了しているよ!")

In [7]:
def answer_writing(selection_position, exchange_operation):
    f = open('answer.txt', 'a')
    f.write(str(selection_position[1]) + str(selection_position[0]) + '\n')
    f.write(str(len(exchange_operation)) + '\n')
    f.write(exchange_operation + '\n')    
    f.close()

    exchange_operation = ""

    return exchange_operation

# ↓それぞれの処理

In [8]:
# 2行を除いた盤面の並び替え(たくと)
def puzzle_board_sort_top(puzzle_board, selection_times, exchange_operation, fixed_value):
    #選択値の初期位置を記憶させておく
    selection_value = colmun * row
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))
    selection_position = selection_coord                #選択画像初期位置

    #回答 選択回数書き込み
    f = open('answer.txt', 'w')
    f.write(str(2) + '\n')
    f.close()

    for i in range(row - 2):
        for j in range(colmun):
            target_value = i * colmun + j + 1           #動かしたい値
            selection_value = colmun * row              #選択値

            target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
            selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

            # print("target_num = " + str(target_value) + "---------------------------------------------------------------------------------------------------")#------------------------------------------

            # print(puzzle_board)#------------------------------------------
            # print("")#------------------------------------------
            
            
            #最終的に置きたい座標
            if target_value % colmun == 0: coord = (target_value // colmun - 1, colmun - 1)
            else: coord = (target_value // colmun, target_value % colmun - 1)
            
            #targetが下端でない
            if target_coord[0] != row-1:
                #最終的に起きたい位置が右2列以外
                if j != colmun-1 and j != colmun-2:
                    # print("not下端, 右2列以外")  #------------------------------------------

                    #targetが最終的に置きたいx座標よりも右
                    if target_coord[1] > coord[1]:
                        #targetを最終的に置きたいy座標に移動
                        if target_coord[0] < selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        
                        #targetの左下にselection(16)を置く
                        if target_coord[1] <= selection_coord[1]:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        

                        #targetを置きたい座標のx座標と同じになるまで移動
                        if target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[1] != coord[1]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                    
                    #targetを最終的に置きたいx座標よりも左
                    elif target_coord[1] < coord[1]:
                        if target_coord[0] < selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        elif target_coord[0] >= selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                        #targetの右下にselection(16)を置く
                        if target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)


                        #targetを置きたい座標のx座標と同じになるまで移動
                        if target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[1] != coord[1]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            #targetの右にselection(16)をおく
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        
                    
                    #targetが最終的に置きたいx座標と同じ
                    elif target_coord[1] == coord[1]:
                        #targetの右にselection(16)をおく
                        #targetよりselectionが右
                        if target_coord[1] < selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりselectionが上
                            if target_coord[0] > selection_coord[0]:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりselectionが下
                            else:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)

                        #targetよりselectionが左
                        elif target_coord[1] > selection_coord[1]:
                            #targetがselectionより上or下
                            if target_coord[0] != selection_coord[0]:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                #targetよりselectionが上
                                if target_coord[0] > selection_coord[0]:
                                    while target_coord[0] != selection_coord[0]:
                                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                #targetよりselectionが下
                                else:
                                    while target_coord[0] != selection_coord[0]:
                                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            #targetがselectionと同じx座標
                            else:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        
                        #targetとselectionのx座標が同じ
                        else:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりselectionが上
                            if target_coord[0] > selection_coord[0]:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりselectionが下
                            else:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)

                         
                    #targetを置きたい座標のy座標と同じになるまで移動
                    if target_coord[0] != coord[0]:
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[0] != coord[0]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                
                #最終的に置きたい位置が右から2列目------------------------------------------------------------------------------------------------------------------
                elif j == colmun-2:
                    # print("not下端, 右から2列目")  #------------------------------------------

                    #最終的に置きたい座標にある
                    if target_coord[0] == coord[0] and target_coord[1] == coord[1]:
                        while target_coord[1]+1 != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[0] != selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)


                    #最終的に置きたいy座標でない
                    elif target_coord[0] != coord[0]:
                        #最終的におきたいx座標でない
                        if target_coord[1] != coord[1]+1:
                            #targetよりも下にselectionがある
                            if target_coord[0] < selection_coord[0]:
                                while target_coord[0]+1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりも上にselectionがある
                            elif target_coord[0] >= selection_coord[0]:
                                while target_coord[0]+1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                            #targetの右にselectionを移動
                            #targetよりも右にselectionがある
                            if target_coord[1] < selection_coord[1]:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりも左にselectionがある
                            else:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            
                            
                            #targetを右端まで移動
                            if target_coord[1] != coord[1]+1:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                while target_coord[1] != coord[1]+1:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                            #targetを最終的に置きたいy座標まで移動
                            if target_coord[0] != coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                while target_coord[0] != coord[0]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                        #最終的におきたいx座標にある
                        elif target_coord[1] == coord[1]+1: 
                            #targetのx座標の左にselectionを移動
                            if target_coord[1] == selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            else:
                                while target_coord[1]-1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                            #targetの左上に移動
                            #targetよりも下にselectionがある
                            if target_coord[0] <= selection_coord[0]:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりも上にselectionがある
                            else:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                            #targetを最終的に置きたいy座標まで移動
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[0] != coord[0]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                #最終的に置きたい位置が右端-----------------------------------------------------------------------------------------------------------------------
                else:
                    # print("not下端, 右端")  #------------------------------------------

                    #逆順に並んでいる
                    if target_coord[0] == coord[0] and target_coord[1] == coord[1]-1:
                        #targetの下にselectionを移動
                        if target_coord[1] < selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                    elif target_coord[0]-1 == selection_coord[0] and target_coord[1] == selection_coord[1] and selection_coord[0] == coord[0]:
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                    #targetがtarget-1
                    elif target_coord[0] == coord[0]+1 and target_coord[1] == coord[1]:
                        #targetの左にselection
                        #targetの左の座標に移動
                        if target_coord[1] > selection_coord[1]:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの左上に移動
                        if target_coord[0] <= selection_coord[0]:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        
                        #3,4を最終的に置きたい座標となるよう移動
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                    elif target_coord[1] == coord[1]:
                        #targetの上にselectionがある
                        if target_coord[0] > selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの下にselectionがある
                        elif target_coord[0] < selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)

                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                        while target_coord[0] != coord[0]:
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)


                    elif target_coord[0] != coord[0]+1 or target_coord[1] != coord[1]:
                        #targetの上にselectionがある
                        if target_coord[0] >= selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの下にselectionがある
                        elif target_coord[0] < selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value) 

                        #targetの右にselectionがある
                        if target_coord[1] < selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの左にselectionがある
                        elif target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                        

                        #targetを右端まで移動
                        # if target_coord[1] != coord[1]:
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                        #targetをtarget-1の下に移動
                        if target_coord[0] != coord[0]+1:
                            while target_coord[0] != coord[0]+1:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)

                        #3,4を最終的に置きたい座標となるよう移動
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)


            #targetが下端
            else:
                #最終的に起きたい位置が右2列以外-------------------------------------------------------------------------------------------------------------------
                if j != colmun-1 and j != colmun-2:
                    # print("下端, 右2列以外")  #------------------------------------------

                    #右2列を除いた列を並び替え
                    if j != row - 1 and j != row - 2:
                        #targetが最終的におきたいx座標よりも右
                        if target_coord[1] > coord[1]:
                            #targetの上の座標にselection(16)を置く
                            if target_coord[0] == selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            else:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            #targetの左上にselection(16)を置く
                            if target_coord[1] <= selection_coord[1]:
                                while target_coord[1]-1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            else:
                                while target_coord[1]-1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            
                            #targetをおきたい座標のx座標と同じになるまで移動
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[1] != coord[1]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            #targetの上にselectionを置く
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                        #targetが最終的におきたいx座標よりも左
                        elif target_coord[1] < coord[1]:
                            #targetの上の座標にselection(16)を置く
                            if target_coord[0] == selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            else:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            #targetの右上にselection(16)を置く
                            if target_coord[1] >= selection_coord[1]:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            else:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                            #targetを置きたい座標のx座標と同じになるまで移動
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[1] != coord[1]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                            #targetの上にchoic(16)を置く
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                        #targetが最終的におきたいx座標と同じ
                        else:
                            #targetの上の座標にselection(16)を置く
                            if target_coord[0] == selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            else:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                            #targetの上にselectionを置く
                            if target_coord[1] < selection_coord[1]:
                                while target_coord[1] != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)   
                            elif target_coord[1] > selection_coord[1]:
                                while target_coord[1] != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)   

                        #targetを置きたい座標のy座標になるまで移動
                        if target_coord[0] != coord[0]:
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[0] != coord[0]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                
                #最終的に置きたい位置が右から2列目------------------------------------------------------------------------------------------------------------------
                elif j == colmun-2:
                    # print("下端, 右から2列目")  #------------------------------------------

                    #targetが最終的におきたいx座標よりも左
                    if target_coord[1] < coord[1]+1:
                        #targetの上の座標にselection(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの右上にselection(16)を置く
                        if target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                        #targetを置きたい座標のx座標と同じになるまで移動
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[1] != coord[1]+1:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                    #targetが最終的におきたいx座標の右と同じ
                    elif target_coord[1] == coord[1]+1:
                        #targetの上の座標にselection(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの上にselection(16)を移動
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                    #targetを最終的に置きたいy座標まで移動
                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                    while target_coord[0] != coord[0]:
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                #最終的に置きたい位置が右端------------------------------------------------------------------------------------------------------------------------
                else:
                    # print("下端, 右端")  #------------------------------------------

                    #targetが最終的におきたいx座標よりも左
                    if target_coord[1] < coord[1]:
                        #targetの上の座標にselection(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの右上にselection(16)を置く
                        if target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                        #targetを置きたい座標のx座標と同じになるまで移動
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                    #targetが最終的におきたいx座標と同じ
                    elif target_coord[1] == coord[1]:
                        #targetの上の座標にselection(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの上にselection(16)を移動
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                    
                    #targetを最終的に置きたいy座標まで移動
                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                    while target_coord[0] != coord[0]+1:
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                    
                    #3,4を最終的におきたい座標となるよう移動
                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
            
            if j != colmun-1 and j != colmun-2:
                fixed_value.append(target_value)
            elif j == colmun-1:
                fixed_value.append(target_value-1)
                fixed_value.append(target_value)

            # print(puzzle_board)#------------------------------------------
            # print("並び替え済み：" + str(fixed_value))  #------------------------------------------
            # print()#------------------------------------------

    return exchange_operation, selection_position, fixed_value


In [28]:
# 下2行の右4マスを除いた盤面の並び替え(たいち)
def puzzle_board_sort_bottom_left(puzzle_board, selection_times, exchange_operation, fixed_value):
    #todo: 並び替えが完了したら fixed_value に値を追加して欲しい
    
    #target_valueの求めかた
    y = 6#行
    x = 6#列
    for i in range(x-2):
        for j in reversed(range(y-2, y)):
            target_value = j * x + i+1
            selection_value = colmun * row

            target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
            selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

            # print("target_value = " + str(target_value))
            
            #一旦置きたい座標(4*4)
            #if target_value == 13: coord = (2, 0)
            #elif target_value == 9: coord = (2, 1)
            #elif target_value == 14: coord = (2, 1)
            #elif target_value == 10: coord = (2, 2)
            
            #一旦起きたい座標(6*6)
            if target_value == 31: coord = (4, 0)
            elif target_value == 25: coord = (4, 1)
            elif target_value == 32: coord = (4, 1)
            elif target_value == 26: coord = (4, 2)
            elif target_value == 33: coord = (4, 2)
            elif target_value == 27: coord = (4, 3)
            elif target_value == 34: coord = (4, 3)
            elif target_value == 28: coord = (4, 4)
            
            #一旦置きたい座標のy座標を揃える
            if target_coord[0] != coord[0]:
                #targetよりselectionの方が上
                if target_coord[0] > selection_coord[0]:
                    #targetよりselectionの方が右
                    if target_coord[1] < selection_coord[1]:
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                    #targetよりselectionの方が左
                    elif target_coord[1] > selection_coord[1]:
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                #targetとselectionのy座標が同じ
                else:
                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                    #targetよりselectionの方が右
                    if target_coord[1] < selection_coord[1]:
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                    #targetよりselectionの方が左
                    elif target_coord[1] > selection_coord[1]:
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                #targetのy座標をcoordと揃える
                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                
              
            #targetが一旦置きたいx座標よりも右
            if target_coord[1] > coord[1]:
                #targetの左にselection(16)を置く
                if target_coord[1] <= selection_coord[1]:
                    while target_coord[1]-1 != selection_coord[1]:
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                else:
                    while target_coord[1]-1 != selection_coord[1]:
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                               
                if target_coord[0] < selection_coord[0]:
                    while target_coord[0] != selection_coord[0]:
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        
#                 elif target_coord[0] > selection_coord[0]:
#                     while target_coord[0] != selection_coord[0]:
#                         target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                #targetを置きたい座標のx座標と同じになるまで移動
                if target_coord[1] != coord[1]:
                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        
                    while target_coord[1] != coord[1]:
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                    
            #targetが一旦置きたいx座標よりも左--------ここバグる
            elif target_coord[1] < coord[1]:
                #targetの右にselection(16)を置く
                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

#                 if target_coord[1] >= selection_coord[1]:
#                     while target_coord[1]+1 != selection_coord[1]:
#                         target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
#                 else:
#                     while target_coord[1]+1 != selection_coord[1]:
#                          target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        
#                 if target_coord[0] < selection_coord[0]:
#                     while target_coord[0] != selection_coord[0]:
#                          target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
#                 elif target_coord[0] > selection_coord[0]:
#                     while target_coord[0] != selection_coord[0]:
#                          target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        
#                 #targetを置きたい座標のx座標と同じになるまで移動
#                 if target_coord[1] != coord[1]:
#                     target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
#                     target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
#                     while target_coord[1] != coord[1]:
#                         target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
#                         target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
#                         target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
#                         target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
#                         target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                
#                         #targetの右にselection(16)をおく---------???????????????????????
#                         target_pos, selection_pos, exchange = down(target_num, selection_num, exchange)
#                         target_pos, selection_pos, exchange = right(target_num, selection_num, exchange)
#                         target_pos, selection_pos, exchange = right(target_num, selection_num, exchange)
#                         target_pos, selection_pos, exchange = up(target_num, selection_num, exchange)      
                    
            #targetが最終的に置きたいx座標と同じ
            elif target_coord[1] == coord[1]:
                #targetの右にselection(16)をおく
                #targetよりselectionが右
                if target_coord[1] < selection_coord[1]:
                    while target_coord[1]+1 != selection_coord[1]:
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                    #targetよりselectionが下
                    if target_coord[0] < selection_coord[0]:
                        while target_coord[0] != selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)

                #targetよりselectionが左
                elif target_coord[1] > selection_coord[1]:
                    #targetがselectionより上or下
                    if target_coord[0] != selection_coord[0]:
                        while target_coord[1]+1 != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        #targetよりselectionが下
                        if target_coord[0] < selection_coord[0]:
                            while target_coord[0] != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        #targetがselectionと同じx座標
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        
                #targetとselectionのx座標が同じ
                else:
                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                    #targetよりselectionが下
                    if target_coord[0] < selection_coord[0]:
                        while target_coord[0] != selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            
            
        #右回りして値を格納する
        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

    return exchange_operation, fixed_value

In [29]:
# 右下4ますの並び替え(しのべー)
def puzzle_board_sort_bottom_right(puzzle_board, selection_times, selection_position, exchange_operation, fixed_value):
    #todo: 並び替えが完了したら fixed_value に値を追加して欲しい
    #todo: 選択画像変更時には exchange_operation = answer_writing() を実行して欲しい
    #todo: 選択値を変更したら、変更時の座標を selectin_position に保存して欲しい
    #todo: この関数に  selection_position(選択値の初期位置) を追加

    print("がんば!")


In [35]:
# 上3つをまとめたもの
def puzzle_board_sort(puzzle_board):
    selection_times = 0                     #選択回数
    exchange_operation = ""                 #交換操作

    f = open('answer.txt', 'w')
    f.write('2\n')    
    f.close()

    fixed_value = []                        #並び替え完了の値
    
    #たくと担当
    exchange_operation, selection_position, fixed_value = puzzle_board_sort_top(puzzle_board, selection_times, exchange_operation, fixed_value)
    print("初期位置：" + str(selection_position[1]) + str(selection_position[0]))
    print(exchange_operation)
    print("交換回数：" + str(len(exchange_operation)))
    print("並び替え済み：" + str(fixed_value))

    print()

    #たいち担当
    exchange_operation, fixed_value = puzzle_board_sort_bottom_left(puzzle_board, selection_times, exchange_operation, fixed_value)
    print(exchange_operation)
    print("交換回数：" + str(len(exchange_operation)))

    answer_writing(selection_position, exchange_operation)

    # #しのべー担当
    # puzzle_board_sort_bottom_right(puzzle_board, selection_times, selection_position, exchange_operation, fixed_value)

# メイン関数

In [36]:
puzzle_board =  np.array([])    #現在の盤面を表す配列

#問題盤面生成
puzzle_board, colmun, row, selection_cost, copy_board = puzzle_generation(puzzle_board, 'issue/4x4.txt')

# puzzle_board = copy_board.copy()

# puzzle_board = copy_230.copy()

print('colmun(列→)：' + str(colmun))
print('row(行↓)：' + str(row))
print('selection_cost(選択コスト)：' + str(selection_cost) + '\n')
print(puzzle_board)
print()

# 並び替え
puzzle_board_sort(puzzle_board)

print()
print(puzzle_board)



# i = 0

# while i != 500:
#     puzzle_board, colmun, row, selection_cost, copy_board = puzzle_generation(puzzle_board, 'issue/4x4.txt')
#     i += 1
#     dynamicvariable_name = "copy_" + str(i)
#     vars()[dynamicvariable_name] = copy_board
#     #並び替え
#     print(str(i) + "回目")
#     puzzle_board_sort(puzzle_board)

colmun(列→)：6
row(行↓)：6
selection_cost(選択コスト)：2

[[22  5 12 14 13 29]
 [25 24 35 20  9 11]
 [18 28 15 21 33 16]
 [ 8 36 26 34 19  7]
 [23 30 10 17 31 27]
 [ 3  4 32  6  2  1]]

初期位置：13
DRRRDRULLDRULLDRULLDRULLDRULDRUULDRUULDRUULDRUULDDDDDRRRRURDLLURDLLURDLLURULDRUULDRUULDRUULDDDDDLURRDLURDRUULDRUULDRUULDRUULDDDDRDLURDRUULDRUULDRUULDRUULDDDDDRRULURDLUURDLUURDLUURDDDDDLURDLUURDLUURDLUURDLUURDDDDLDRULLDRULLDRULLDRULLDRULDRUULDRUULDRUULDDDRDLURDRUULDRUULDRUULDRURDDDLDLURRDLURDRUULDRUULDRUULDRURDDDDLUURDLUURDLUURDDDLLLURDLLURDLLURULDDDRRRURDLLURDLLURULDRUULDRRUDDURULDDULDRRULDRRULURDDDLLLULDRRULDRRULDRRULURDLUURDLDLURDLLURDLLURDLLURDDRURDLLURULDRRURDLLURDDLURDRUULDDLULDRRULDRRULDRRULURDLLLDLURRDLURRDLURRDLURDLUURD
交換回数：533
並び替え済み：[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

DRRRDRULLDRULLDRULLDRULLDRULDRUULDRUULDRUULDRUULDDDDDRRRRURDLLURDLLURDLLURULDRUULDRUULDRUULDDDDDLURRDLURDRUULDRUULDRUULDRUULDDDDRDLURDRUULDRUULDRUULDRUULDDDDDRRULURDLUURDLUURDLU